In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
import gc
#pd.options.display.max_colwidth=100
pd.set_option('display.max_colwidth',100)
# Any results you write to the current directory are saved as output.

In [ ]:
gc.collect()

### Loading dataset and basic visualization

In [ ]:
train=pd.read_csv('../input/train.tsv',sep='\t')
print(train.shape)
train.head()

In [ ]:
test=pd.read_csv('../input/test.tsv',sep='\t')
print(test.shape)
test.head()

In [ ]:
sub=pd.read_csv('../input/sampleSubmission.csv')
sub.head()

**Adding Sentiment column to test datset and joing train and test for preprocessing**

In [ ]:
test['Sentiment']=-999
test.head()

In [ ]:
df=pd.concat([train,test],ignore_index=True)
print(df.shape)
df.tail()

In [ ]:
del train,test
gc.collect()

** cleaning review**

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer,WordNetLemmatizer
stemmer=SnowballStemmer('english')
lemma=WordNetLemmatizer()
from string import punctuation
import re
from bs4 import BeautifulSoup

In [ ]:
def clean_review(review_col):
    review_corpus=[]
    for i in range(0,len(review_col)):
        review=str(review_col[i])
        review=re.sub('[^a-zA-Z]',' ',review)
        #review=[stemmer.stem(w) for w in word_tokenize(str(review).lower())]
        review=[lemma.lemmatize(w) for w in word_tokenize(str(review).lower())]
        review=' '.join(review)
        review_corpus.append(review)
    return review_corpus

In [ ]:
df['clean_review']=clean_review(df.Phrase.values)
df.head()


** seperating train and test dataset**

In [ ]:
df_train=df[df.Sentiment!=-999]
df_train.shape

In [ ]:
df_test=df[df.Sentiment==-999]
df_test.drop('Sentiment',axis=1,inplace=True)
print(df_test.shape)
df_test.head()

In [ ]:
del df
gc.collect()

### Bag of Words model

In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer
# cv=CountVectorizer()

In [ ]:
# c_train=cv.fit_transform(df_train.clean_review).toarray()
# print(c_train.shape)
# c_test=cv.transform(df_test.clean_review).toarray()
# print(c_test.shape)

In [ ]:
# bow_df=pd.DataFrame(data=c_train,columns=cv.get_feature_names())
# bow_df.head()

### Tfidf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(ngram_range=(1,2),max_df=0.95,min_df=10,sublinear_tf=True)

In [ ]:
c2_train=tfidf.fit_transform(df_train.clean_review).toarray()
print(c2_train.shape)
c2_test=tfidf.transform(df_test.clean_review).toarray()
print(c2_test.shape)

### One hot encoding of target variable

In [ ]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

In [ ]:
le=LabelEncoder()
y=le.fit_transform(df_train.Sentiment.values)
#y=pd.get_dummies(y).values
y.shape

In [ ]:
del df_train,df_test
gc.collect()

### splitting data into train and validation set

In [ ]:
# from sklearn.model_selection import train_test_split

In [ ]:
# X_train,X_val,y_train,y_val=train_test_split(c2_train,y,test_size=0.2)
# print(X_train.shape,y_train.shape)
# print(X_val.shape,y_val.shape)

## 1. Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()

In [ ]:
lr.fit(c2_train,y)

In [ ]:
y_pred=lr.predict(c2_test)

In [ ]:
sub.Sentiment=y_pred
sub.head()

In [ ]:
sub.to_csv('submission.csv',index=False)